In [1]:
from game import controlled_run

pygame 2.0.1 (SDL 2.0.14, Python 3.7.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import numpy as np
from game import DO_NOTHING
from game import JUMP

import matplotlib.pyplot as plt

import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from game import controlled_run

import random
import numpy as np
from collections import deque


#games_count=0
#total_number_of_games = 1000

# model = Sequential()
# model.add(Dense(1,input_dim=1,activation='sigmoid'))
# model.add(Dense(2,activation='softmax'))
# model.compile(Adam(lr=0.1),loss='categorical_crossentropy',metrics=['accuracy'])

# x_train = np.array([])
# y_train = np.array([])


# flaw in game design, it should be based on whether or not you make it past an opponent
class Wrapper(object):
    
    
    def __init__(self):
        controlled_run(self,0)
        
    def control(self,values):
        global x_train
        global y_train
        
        print(values)
        
        if(values['closest_enemy'] == -1):
            return DO_NOTHING
        
        if(values['old_closest_enemy']!=-1):
            print(values['closest_enemy'])
            if(values['score_increased']==1):
                x_train = np.append(x_train,[values['old_closest_enemy']/1000])
                y_train = np.append(y_train,[values['action']])
                
        prediction=model.predict_classes(np.array([[values['closest_enemy']]])/1000)
            
        
        return prediction
    def reward(self,values):
        print("No reward")
    def gameover(self,score):
        global games_count
        global x_train
        global y_train
        
        print(x_train)
        print(y_train)
        
        if(games_count >0):
            y_train_cat = to_categorical(y_train,num_classes=2)
            model.fit(x_train,y_train_cat,epochs=50, verbose =1,shuffle=1)
            
        games_count+=1
        
        if games_count>total_number_of_games: 
            return
        controlled_run(self,games_count)



In [3]:
# w=Wrapper()

In [4]:
GAMMA = 0.95
LEARNING_RATE = 0.0001

MEMORY_SIZE = 1000000
BATCH_SIZE = 64

EXPLORATION_MAX = .35
EXPLORATION_MIN = 0.0
EXPLORATION_DECAY = 0.95

import numpy as np
from game import DO_NOTHING
from game import JUMP

games_count = 0
total_number_of_games = 100

class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(32, input_shape=(observation_space,), activation="relu", kernel_initializer='he_uniform'))
        #self.model.add(BatchNormalization())
        #self.model.add(Dense(64, activation="relu"))
        #self.model.add(BatchNormalization())
        #self.model.add(Dense(32, activation="relu"))
        self.model.add(Dense(128, activation="relu", kernel_initializer='he_uniform'))
        self.model.add(Dense(64, activation="relu", kernel_initializer='he_uniform'))
        #self.model.add(BatchNormalization())
        self.model.add(Dense(32, activation="relu", kernel_initializer='he_uniform'))
        self.model.add(Dropout(0.2))
        self.model.add(BatchNormalization())
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        # self.memory.append((gamenumber,state,action,reward,next_state,done))

    def act(self, state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
    # fit on past data
    def experience_replay(self):
        print("memory: " + str(len(self.memory)))
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        # gamenumber = random(0,10)
        # self.memory(gamenumber)
        cul_reward = 0
        index=0
        lastmove = len(batch)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            cul_reward+=reward
            if not terminal:
                
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            #print(np.argmax(q_values[0]))
            #update the q value for that action 
            q_values[0][action] = q_update
            #Fitting during experience replay?
            self.model.fit(np.array(state), q_values, verbose=0)
            index+=1
        decay = cul_reward/500

        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

In [5]:


# flaw in game design, it should be based on whether or not you make it past an opponent
class Wrapper(object):
    
    
    def __init__(self,dqn_solver):
        self.dqn_solver = dqn_solver
        self.scores=[]
        self.rewards=[]
        controlled_run(self,0)
        
    def control(self,values):
        dqn_solver = self.dqn_solver
        state = [[values['closest_enemy']/1000]]
        step = 0 

        self.old_state = state
        action = dqn_solver.act(state)
        self.action = action
        
        return action
    
    def get_scores(self):
        return self.scores
    
    def get_rewards(self):
        return self.rewards
        
    def reward(self,values):
        
        dqn_solver = self.dqn_solver

        state_next = values['closest_enemy']
        reward = values['score_increased']
        self.rewards.append(reward)
        terminal=False
        try:
            self.state = state_next
            dqn_solver.remember(self.old_state, self.action, reward, [state_next], terminal)
            print("remembered")
        except:
            pass
    
    def gameover(self,score):
        global games_count
        dqn_solver = self.dqn_solver
        reward = -1
        dqn_solver.remember(self.old_state, self.action, reward, self.state, True)
        try:
            print("Run: " + str(games_count) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(score))
        except:
            print("Run: " + str(games_count) + ", score: " + str(score))

        if(games_count >0):
            print("replaying...")
            dqn_solver.experience_replay()

        games_count+=1
        self.scores.append(score)
        if games_count>total_number_of_games: 
            return
        controlled_run(self,games_count)

In [6]:
dqn_solver = DQNSolver(1,2)
game = Wrapper(dqn_solver)

remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 0, exploration: 0.35, score: 6
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1, exploration: 0.35, score: 3
replaying...
memory: 20
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 2, exploration: 0.35, score: 3
replaying...
memory: 31
remembered
remembered
remembered
remembered
Run: 3, exploration: 0.35, score: 1
replaying...
memory: 36
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 4, exploration: 0.35, score: 13
replaying...
memory: 68
remembered
remembered
remember

memory: 478
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 47, exploration: 0.117832887274972, score: 7
replaying...
memory: 488
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 48, exploration: 0.11488706509309769, score: 8
replaying...
memory: 504
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 49, exploration: 0.11201488846577025, score: 10
replaying...
memory: 519
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 50, exploration: 0.109214516254126, score: 5
replaying...
memory: 528
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 51, exploration: 0.10648415334777285, score: 5
replaying...
memory: 536
remembered
remembered
r

Run: 88, exploration: 0.041730708965323604, score: 14
replaying...
memory: 972
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 89, exploration: 0.04068744124119051, score: 13
replaying...
memory: 997
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 90, explorati

remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 122, exploration: 0.017644635726440948, score: 12
replaying...
memory: 1513
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 123, exploration: 0.017203519833279925, score: 4
replaying...
memory: 1526
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 124, exploration: 0.016773431837447925, score: 6
replaying...
memory: 1545
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 125, exploration: 0.016354096041511727, score: 4
replaying...
memory: 1556
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 126, exploration: 0.015945243640473932, score: 

remembered
remembered
remembered
remembered
remembered
Run: 149, exploration: 0.008907120354973368, score: 18
replaying...
memory: 2084
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 150, exploration: 0.008684442346099034, score: 13
replaying...
memory: 2108
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 151, exploration: 0.008467331287446558, score: 3
replaying...
memory: 2118
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 152, exploration: 0.008255648005260394, score: 5
replaying...
memory: 2129
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembe

...

remembered
remembered
remembered
remembered
remembered
Run: 1300, exploration: 1.968112817145466e-15, score: 1
replaying...
memory: 16207
remembered
remembered
remembered
remembered
remembered
Run: 1301, exploration: 1.9189099967168294e-15, score: 0
replaying...
memory: 16213
remembered
remembered
remembered
remembered
Run: 1302, exploration: 1.8709372467989086e-15, score: 0
replaying...
memory: 16218
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1303, exploration: 1.824163815628936e-15, score: 11
replaying...
memory: 16253
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remem

replaying...
memory: 16664
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1343, exploration: 6.62595473506084e-16, score: 2
replaying...
memory: 16672
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1344, exploration: 6.460305866684319e-16, score: 4
replaying...
memory: 16690
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1345, exploration: 6.298798220017211e-16, score: 3
replaying...
memory: 16705
remembered
remembered
remembered
Run: 1346, exploration: 6.14132826451678e-16, score: 0
replaying...
memory: 16709
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1347, exploration: 5.98779505790386e-16, score: 2
replaying...
mem

remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1377, exploration: 2.801595288907848e-16, score: 4
replaying...
memory: 17194
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1378, exploration: 2.7315554066851514e-16, score: 1
replaying...
memory: 17202
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1379, exploration: 2.6632665215180226e-16, score: 7
replaying...
memory: 17229
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1380, exploration:

Run: 1417, exploration: 1.0176302923683886e-16, score: 2
replaying...
memory: 17663
remembered
remembered
remembered
Run: 1418, exploration: 9.921895350591789e-17, score: 0
replaying...
memory: 17667
remembered
remembered
remembered
Run: 1419, exploration: 9.673847966826994e-17, score: 0
replaying...
memory: 17671
remembered
remembered
remembered
Run: 1420, exploration: 9.432001767656319e-17, score: 0
replaying...
memory: 17675
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1421, exploration: 9.196201723464911e-17, score: 2
replaying...
memory: 17685
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 1422, exploration: 8.966296680378288e-17, score: 1
replaying...
memory: 17696
remembered
remembered
remembered
remembered
Run: 1423, exploration: 8.74213926336883e-17, score: 0
replaying...
memory: 17701
remembered
remembered
remembered
remembered
remembered
remembered
r

RecursionError: maximum recursion depth exceeded

In [7]:
scores = game.get_scores()

NameError: name 'game' is not defined

In [ ]:
plt.plot(scores)
plt.xlabel('Round')
plt.ylabel('Score')
plt.title('Total game score by round')
plt.show()

In [ ]:
# game.get_rewards()

In [ ]:
gamma = 0.95
alpha = 0.5
learning_rate_adam = 0.01
epsilon = 0.999
epsilon_decay = 0.99
class DQN:

    def __init__(self, observation_space, action_space):
        
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.action_space = action_space
        self.observation_space = observation_space
        
        self.memory = []
        self.batch_size = 20

        self.model = Sequential()
        self.model.add(Dense(64, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(32, activation="relu"))
        self.model.add(Dense(16, activation="relu"))
        self.model.add(Dropout(0.4))
        self.model.compile(loss="mse", optimizer=Adam(lr=learning_rate_adam))

    def act(self, state):
        if np.random.rand() < self.epsilon:
            return random.randrange(self.action_space)
        q = self.model.predict(state)
        return np.argmax(q[0])

    def experience_replay(self):
        print(self.memory,self.batch_size)
        mem_sample = random.sample(self.memory, min(len(self.memory),self.batch_size))
        for state, action, reward, next_state, done in mem_sample:
            update_value = reward
            print(update_value)
            if not done:
                update_value = self.alpha * (reward + self.gamma * np.max(self.model.predict(next_state)[0]))
            q = self.model.predict(state)
            q[0][action] = update_value
            # print(state,type(state))
            # print(q,type(q))
            self.model.fit(np.array(state), q, verbose=0)
        self.epsilon *= epsilon_decay
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

In [ ]:
# dqn_solver = DQN(1,2)
# game = Wrapper(dqn_solver)